In [ ]:
import os
import time
import random
import pandas as pd
import requests
from youtube_transcript_api import YouTubeTranscriptApi
from fake_useragent import UserAgent
import logging
import sys 
from datetime import datetime
import threading
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import re 
import json 
from urllib.parse import urlencode 

# Define the base directory for all file operations
BASE_DIR = "C:\\Users\\Pooja\\Downloads"

# ===== FIXED CONFIGURATION WITH MONITORING (Windows Paths) =====
INPUT_CSV = os.path.join(BASE_DIR, "youtube_data_metadata.xls - youtube_data_metadata.xls.csv") 
OUTPUT_FOLDER = os.path.join(BASE_DIR, "Output")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
OUTPUT_CSV = os.path.join(OUTPUT_FOLDER, "all_video_details_and_transcripts.csv") 
PROGRESS_LOG = os.path.join(OUTPUT_FOLDER, "progress_log.txt") 

# ==============================================================================
# 💥 PROXY RELIABILITY SETTINGS (Adjusted Cooldown for Flow Test) 💥
# Cooldown reduced to 5 minutes to prevent very long initial waits that might be causing the script to bypass the main loop.
# ==============================================================================
PROXIES = [
    # General high-anonymity proxies (often stable)
    "http://49.148.236.124:8080", "http://103.156.249.52:8080", "http://103.144.18.74:8080", 
    "http://185.82.99.148:9091", "http://103.165.155.69:8080", "http://103.117.192.174:80", 
    "http://190.107.224.150:3128", "http://103.144.18.98:8080", "http://134.122.116.174:8080", 
    "http://147.182.211.215:8080", "http://68.183.111.90:8080", "http://164.90.179.64:8080",
]

# Optimized settings
RETRIES = 5 
MIN_DELAY = 10 # Reduced minimum delay
MAX_DELAY = 30 # Reduced maximum delay
REQUESTS_PER_PROXY = 1 
COOLDOWN_TIME = 300 # 5 minutes (300 seconds) for faster testing
REQUEST_TIMEOUT = 45 
# ==============================================================================

# Global monitoring variables
monitoring_data = {
    'total_videos': 0,
    'processed_videos': 0,
    'successful_transcripts': 0,
    'failed_videos': 0,
    'current_video': '',
    'start_time': 0,
    'eta_minutes': 0,
    'current_status': 'Initializing...',
    'proxy_stats': {},
    'transcript_types': {'manual': 0, 'auto-generated': 0, 'unknown': 0},
    'recent_errors': []
}

# --- Utility Functions (Only changed the Proxy Manager logic for faster start) ---

def fetch_additional_proxies():
    return []

extra_proxies = fetch_additional_proxies()
PROXIES.extend(extra_proxies)
PROXIES = list(set([p for p in PROXIES if p.startswith('http://') and ':' in p]))

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.handlers.clear() 

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler(PROGRESS_LOG, encoding='utf-8')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setFormatter(formatter)
stream_handler.encoding = 'utf-8' 
logger.addHandler(stream_handler)


ua = UserAgent()

class MonitoringProxyManager:
    def __init__(self, proxies):
        self.proxies = proxies
        self.request_counts = {proxy: 0 for proxy in proxies}
        self.failed_proxies = set()
        self.last_used = {proxy: 0 for proxy in proxies} 
        self.success_rates = {proxy: 1.0 for proxy in proxies}

    def get_next_proxy(self):
        available_proxies = [p for p in self.proxies if p not in self.failed_proxies]

        if not available_proxies:
            logger.warning("⚠️ All proxies failed, resetting failed list...")
            self.failed_proxies.clear()
            available_proxies = self.proxies

        current_time = time.time()
        ready_proxies = []
        
        # 💡 FIX: Temporarily prioritize proxies that haven't been used at all
        never_used_proxies = [p for p in available_proxies if self.last_used.get(p, 0) == 0]
        if never_used_proxies:
             ready_proxies = never_used_proxies
        else:
             # If all have been used, check cooldown
             for p in available_proxies:
                time_since_last = current_time - self.last_used.get(p, 0)
                if time_since_last >= COOLDOWN_TIME or self.request_counts[p] < REQUESTS_PER_PROXY:
                     ready_proxies.append(p)

        if not ready_proxies:
            # If all are cooling down, wait for the earliest one
            wait_proxy = min(available_proxies, key=lambda p: self.last_used.get(p, 0))
            time_since_last = current_time - self.last_used.get(wait_proxy, 0)
            cooldown_remaining = COOLDOWN_TIME - time_since_last
            if cooldown_remaining > 0:
                monitoring_data['current_status'] = f"😴 All proxies cooling down. Waiting {cooldown_remaining:.0f}s for next..."
                update_progress_display()
                logger.info(f"🕒 All proxies cooling down, waiting {cooldown_remaining:.0f}s...")
                time.sleep(cooldown_remaining)
                ready_proxies = [wait_proxy] 

        if not ready_proxies: 
            ready_proxies = available_proxies

        def proxy_score(proxy):
            if current_time - self.last_used.get(proxy, 0) >= COOLDOWN_TIME:
                self.request_counts[proxy] = 0
            
            usage_factor = 1 / (self.request_counts[proxy] + 1)
            success_factor = self.success_rates[proxy]
            return usage_factor * success_factor

        best_proxy = max(ready_proxies, key=proxy_score)
        
        self.request_counts[best_proxy] += 1
        self.last_used[best_proxy] = current_time

        return best_proxy

    def mark_success(self, proxy):
        self.success_rates[proxy] = min(1.0, self.success_rates[proxy] + 0.1)
        if proxy in self.failed_proxies:
            self.failed_proxies.remove(proxy)
        self._update_monitoring()

    def mark_failure(self, proxy):
        self.success_rates[proxy] = max(0.0, self.success_rates[proxy] - 0.5) 
        if self.success_rates[proxy] <= 0.0:
            self.failed_proxies.add(proxy)
            logger.error(f"💀 PROXY REMOVED: {proxy} due to repeated failures.")
        self._update_monitoring()

    def _update_monitoring(self):
        monitoring_data['proxy_stats'] = {
            'total': len(self.proxies),
            'working': len([p for p in self.proxies if p not in self.failed_proxies]),
            'failed': len(self.failed_proxies),
            'avg_success_rate': sum(self.success_rates.values()) / max(1, len(self.success_rates))
        }

def update_progress_display():
    clear_output(wait=True)

    if monitoring_data['total_videos'] > 0:
        progress_percent = (monitoring_data['processed_videos'] / monitoring_data['total_videos']) * 100
        success_rate = (monitoring_data['successful_transcripts'] / max(1, monitoring_data['processed_videos'])) * 100
    else:
        progress_percent = 0
        success_rate = 0

    bar_length = 40
    filled_length = int(bar_length * progress_percent / 100)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)

    print("🎯 YOUTUBE TRANSCRIPT & METADATA EXTRACTION - LIVE MONITOR")
    print("=" * 60)
    print(f"📊 Progress: [{bar}] {progress_percent:.1f}%")
    print(f"📹 Videos: {monitoring_data['processed_videos']}/{monitoring_data['total_videos']}")
    print(f"✅ Success (Transcripts): {monitoring_data['successful_transcripts']} ({success_rate:.1f}%)")
    print(f"❌ Failed (Transcripts): {monitoring_data['failed_videos']}")
    print(f"🕐 Runtime: {(time.time() - monitoring_data['start_time'])/60:.1f} minutes")
    print(f"⏱️ ETA: {monitoring_data['eta_minutes']:.1f} minutes remaining")
    print(f"📺 Current: {monitoring_data['current_video']}")
    print(f"🔄 Status: {monitoring_data['current_status']}")

    if monitoring_data['proxy_stats']:
        stats = monitoring_data['proxy_stats']
        print(f"🌐 Proxies: {stats['working']}/{stats['total']} working (Success: {stats['avg_success_rate']:.2f}) - Cooldown: {COOLDOWN_TIME/60:.0f} mins")

    if sum(monitoring_data['transcript_types'].values()) > 0:
        types = monitoring_data['transcript_types']
        print(f"📝 Types: Manual={types['manual']}, Auto={types['auto-generated']}, Unknown={types['unknown']}")

    if monitoring_data['recent_errors']:
        print(f"⚠️ Recent errors: {len(monitoring_data['recent_errors'])}")
        for error in monitoring_data['recent_errors'][-3:]:
            print(f"   • {error}")

    print("=" * 60)

def fetch_video_metadata_requests(video_id, proxy):
    url = f"https://www.youtube.com/watch?v={video_id}"
    
    metadata = {
        'title': None, 'channel_name': None, 'view_count': None, 
        'publish_date': None, 'video_length': None
    }
    
    try:
        session = requests.Session()
        session.proxies = {"http": proxy, "https": proxy}
        headers = {'User-Agent': ua.random, 'Accept-Language': 'en-US,en;q=0.5'}
        
        response = session.get(url, headers=headers, timeout=REQUEST_TIMEOUT) 
        response.raise_for_status()
        html_content = response.text

        match = re.search(r'var ytInitialPlayerResponse\s*=\s*(\{.*?\})\s*;', html_content)
        if match:
            data = json.loads(match.group(1))
            
            details = data.get('videoDetails', {})
            metadata['title'] = details.get('title')
            metadata['channel_name'] = details.get('author')
            metadata['view_count'] = int(details.get('viewCount', 0)) if details.get('viewCount') else None
            metadata['video_length'] = int(details.get('lengthSeconds', 0)) if details.get('lengthSeconds') else None

            microformat = data.get('microformat', {}).get('playerMicroformatRenderer', {})
            metadata['publish_date'] = microformat.get('publishDate')
        else:
            title_match = re.search(r'"title":"([^"]+)"', html_content)
            if title_match:
                metadata['title'] = title_match.group(1)
            
            logger.warning(f"⚠️ Metadata: Could not find full ytInitialPlayerResponse for {video_id}")

        return metadata

    except requests.exceptions.HTTPError as e:
        if response.status_code in [404, 410]:
             logger.warning(f"⚠️ Metadata: Video is unavailable (404/410)")
             return metadata
        raise e
    except Exception as e:
        logger.error(f"❌ Metadata fetch error for {video_id}: {e}")
        raise e

def fetch_transcript_fixed(video_id, proxy):
    try:
        monitoring_data['current_status'] = f"🔍 Fetching transcript via {proxy[:20]}..."
        update_progress_display()

        session = requests.Session()
        session.proxies = {"http": proxy, "https": proxy}
        headers = {
            'User-Agent': ua.random, 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5', 'Connection': 'keep-alive', 'Referer': 'https://www.youtube.com/',
        }

        import youtube_transcript_api._api as _api
        _api.requests = session

        original_get = session.get
        def enhanced_get(*args, **kwargs):
            kwargs.setdefault('headers', {}).update(headers)
            kwargs.setdefault('timeout', REQUEST_TIMEOUT) 
            return original_get(*args, **kwargs)
        session.get = enhanced_get
        ytt_api = YouTubeTranscriptApi()

        try:
            transcript_list = ytt_api.list_transcripts(video_id)

            try:
                manual_transcript = transcript_list.find_manually_created_transcript(['en'])
                fetched_transcript = manual_transcript.fetch()
                transcript_text = " ".join([snippet['text'] for snippet in fetched_transcript])
                monitoring_data['transcript_types']['manual'] += 1
                logger.info("✅ Found MANUAL English transcript")
                return transcript_text, "manual"
            except:
                pass

            try:
                auto_transcript = transcript_list.find_generated_transcript(['en'])
                fetched_transcript = auto_transcript.fetch()
                transcript_text = " ".join([snippet['text'] for snippet in fetched_transcript])
                monitoring_data['transcript_types']['auto-generated'] += 1
                logger.info("✅ Found AUTO-GENERATED English transcript")
                return transcript_text, "auto-generated"
            except:
                pass

        except Exception as e:
            try:
                fetched_transcript = ytt_api.fetch(video_id) 
                transcript_text = " ".join([snippet['text'] for snippet in fetched_transcript])
                monitoring_data['transcript_types']['unknown'] += 1
                logger.info("✅ Found transcript using fallback method")
                return transcript_text, "unknown"
            except:
                pass

        return None, None

    except Exception as e:
        raise e

def get_video_details_with_retry(video_id, proxy_manager):
    monitoring_data['current_status'] = f"🎬 Processing {video_id[:11]}..."
    update_progress_display()
    
    details = {
        "video_id": video_id,
        'title': None, 'channel_name': None, 'view_count': None, 
        'publish_date': None, 'video_length': None, "transcript": None,
        "transcript_type": None, "is_transcript_available": False 
    }
    
    # === 1. Metadata Fetching Loop ===
    for attempt in range(1, RETRIES + 1):
        proxy = proxy_manager.get_next_proxy()
        proxy_display = proxy[:30] + "..." if len(proxy) > 30 else proxy
        monitoring_data['current_status'] = f"🔄 Metadata Attempt {attempt}/{RETRIES} via {proxy_display}"
        update_progress_display()
        logger.info(f"🔄 Metadata Attempt {attempt}/{RETRIES} with proxy: {proxy_display}")

        try:
            metadata = fetch_video_metadata_requests(video_id, proxy)
            proxy_manager.mark_success(proxy)
            details.update(metadata)
            logger.info("✅ Metadata fetched successfully.")
            break
        except Exception as e:
            error_msg = str(e)
            proxy_manager.mark_failure(proxy)
            logger.warning(f"⚠️ Metadata failed: {error_msg[:100]}...")
            
            if attempt < RETRIES:
                delay = random.uniform(5, 15)
                time.sleep(delay)
    
    # === 2. Transcript Fetching Loop ===
    for attempt in range(1, RETRIES + 1):
        proxy = proxy_manager.get_next_proxy()
        proxy_display = proxy[:30] + "..." if len(proxy) > 30 else proxy
        monitoring_data['current_status'] = f"🔄 Transcript Attempt {attempt}/{RETRIES} via {proxy_display}"
        update_progress_display()
        logger.info(f"🔄 Transcript Attempt {attempt}/{RETRIES} with proxy: {proxy_display}")

        try:
            transcript_text, transcript_type = fetch_transcript_fixed(video_id, proxy)

            if transcript_text:
                proxy_manager.mark_success(proxy)
                monitoring_data['successful_transcripts'] += 1
                monitoring_data['current_status'] = f"✅ Success ({transcript_type})"
                logger.info(f"✅ Transcript fetched successfully ({transcript_type})")
                details['transcript'] = transcript_text
                details['transcript_type'] = transcript_type
                details['is_transcript_available'] = True 
                return details
            else:
                proxy_manager.mark_success(proxy) 
                monitoring_data['current_status'] = "⚠️ No transcript available"
                logger.warning("⚠️ No transcript available for this video")
                break 

        except Exception as e:
            error_msg = str(e)
            short_error = error_msg[:50] + "..." if len(error_msg) > 50 else error_msg
            monitoring_data['recent_errors'].append(f"{video_id}: {short_error}")
            if len(monitoring_data['recent_errors']) > 10:
                monitoring_data['recent_errors'] = monitoring_data['recent_errors'][-10:]

            monitoring_data['current_status'] = f"❌ Error: {short_error}"
            logger.warning(f"⚠️ Transcript Failed: {error_msg[:100]}...")

            if any(phrase in error_msg.lower() for phrase in ["blocking requests", "too many requests", "rate limit", "forbidden", "refused it"]):
                proxy_manager.mark_failure(proxy) 

            if attempt < RETRIES:
                delay = random.uniform(15, 30) 
                time.sleep(delay)
    
    if not details['is_transcript_available']:
        monitoring_data['failed_videos'] += 1
        monitoring_data['current_status'] = "❌ Final: No transcript found"
        logger.info(f"❌ Final: No transcript for video {video_id}")

    return details

def save_progress_report():
    report_file = os.path.join(OUTPUT_FOLDER, "progress_report.txt") 
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(f"YOUTUBE TRANSCRIPT EXTRACTION REPORT\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"=" * 50 + "\n")
        f.write(f"Total Videos: {monitoring_data['total_videos']}\n")
        f.write(f"Processed: {monitoring_data['processed_videos']}\n")
        f.write(f"Successful (Transcript): {monitoring_data['successful_transcripts']}\n")
        f.write(f"Failed (No Transcript): {monitoring_data['failed_videos']}\n")
        f.write(f"Success Rate (Transcript): {(monitoring_data['successful_transcripts']/max(1,monitoring_data['processed_videos']))*100:.1f}%\n")
        f.write(f"Runtime: {(time.time() - monitoring_data['start_time'])/60:.1f} minutes\n")
        f.write(f"\nTranscript Types:\n")
        for t_type, count in monitoring_data['transcript_types'].items():
            f.write(f"  {t_type}: {count}\n")
        f.write(f"\nProxy Statistics:\n")
        if monitoring_data['proxy_stats']:
            stats = monitoring_data['proxy_stats']
            f.write(f"  Total: {stats['total']}\n")
            f.write(f"  Working: {stats['working']}\n")
            f.write(f"  Failed: {stats['failed']}\n")
            f.write(f"  Avg Success Rate: {stats['avg_success_rate']:.2f}\n")

# MAIN EXECUTION WITH MONITORING
monitoring_data['start_time'] = time.time()
monitoring_data['current_status'] = "🚀 Starting extraction..."
update_progress_display()

logger.info("🚀 Starting YouTube details and transcript extraction with live monitoring...")
logger.info(f"📁 Input CSV: {INPUT_CSV}")
logger.info(f"📁 Output CSV: {OUTPUT_CSV}")

# Load input data
try:
    df_original = pd.read_csv(INPUT_CSV)
except FileNotFoundError:
    logger.error(f"❌ Input file not found: {INPUT_CSV}")
    print(f"ERROR: Please ensure the file is named '{os.path.basename(INPUT_CSV)}' and is in '{BASE_DIR}'")
    exit()

print(f"📊 Input CSV columns: {list(df_original.columns)}")

# Find video_id column (flexible column name detection, prioritizing 'Video_ID' based on your output)
video_id_column = None
for col in df_original.columns:
    if col.lower() == 'video_id': # Exact match for your column name
        video_id_column = col
        break
    if 'video_id' in col.lower() or 'videoid' in col.lower() or 'id' in col.lower():
        video_id_column = col
        break

if video_id_column is None:
    # If no video_id column found, check for URL and try to extract (omitted detailed URL logic for brevity, 
    # as the previous output shows you have a 'Video_ID' column, we assume this is the main path)
    if not df_original.empty and len(df_original.columns) > 0:
        video_id_column = df_original.columns[0]
        logger.warning(f"⚠️ No standard ID column found. Assuming the first column ('{video_id_column}') contains Video IDs.")
    else:
        raise ValueError("No video identifier column found in CSV.")

# Temporarily rename the original ID column to 'video_id' for processing/merging consistency
original_id_column_name = video_id_column 
if original_id_column_name != 'video_id':
    df_original.rename(columns={original_id_column_name: 'video_id'}, inplace=True)

video_ids = [str(x).strip() for x in df_original['video_id'].dropna().unique().tolist() if str(x).strip() != 'nan']
monitoring_data['total_videos'] = len(video_ids)
logger.info(f"📊 Total unique videos found: {len(video_ids)}")

# Resume support
if os.path.exists(OUTPUT_CSV):
    processed_df = pd.read_csv(OUTPUT_CSV)
    
    id_col_in_processed_df = original_id_column_name if original_id_column_name in processed_df.columns else 'video_id'
    processed_ids = set(processed_df[id_col_in_processed_df].astype(str))
    
    if 'video_id' not in processed_df.columns and id_col_in_processed_df != 'video_id':
         processed_df.rename(columns={id_col_in_processed_df: 'video_id'}, inplace=True)

    scraped_cols_names = ['video_id', 'title', 'channel_name', 'view_count', 'publish_date', 'video_length', 'transcript', 'transcript_type', 'is_transcript_available']
    results_cols = [col for col in processed_df.columns if col in scraped_cols_names]
    results = processed_df[processed_df['video_id'].isin(video_ids)][results_cols].to_dict("records")
    
    monitoring_data['successful_transcripts'] = processed_df['is_transcript_available'].sum()
    monitoring_data['failed_videos'] = len(processed_df) - monitoring_data['successful_transcripts']
    
    logger.info(f"♻️ Resuming: {len(processed_ids)} already processed")
else:
    processed_ids = set()
    results = []

remaining = [vid for vid in video_ids if vid not in processed_ids]
logger.info(f"⏳ Remaining videos to process: {len(remaining)}")

if len(remaining) == 0:
    logger.info("🎉 All videos already processed! Bypassing processing loop.")
    # Exit here is the correct behavior if all are processed. 
    # The previous 0.0 minute run likely happened because len(remaining) was 0.
    save_progress_report()
    exit()

# Initialize proxy manager with monitoring
proxy_manager = MonitoringProxyManager(PROXIES)
logger.info(f"🌐 Initialized with {len(PROXIES)} proxies")

# Main processing loop with enhanced monitoring
processed_count = len(processed_ids)
monitoring_data['processed_videos'] = processed_count

try:
    for idx, vid in enumerate(remaining, start=1):
        monitoring_data['current_video'] = vid
        monitoring_data['current_status'] = f"🎬 Starting {vid[:11]}..."
        update_progress_display()

        logger.info(f"\n📹 [{idx}/{len(remaining)}] Processing video: {vid}")

        video_details = get_video_details_with_retry(vid, proxy_manager)

        results = [res for res in results if res.get('video_id') != vid]
        results.append(video_details)

        pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
        monitoring_data['current_status'] = "💾 Progress saved"
        update_progress_display()
        logger.info("💾 Progress saved to CSV")

        processed_count += 1
        monitoring_data['processed_videos'] = processed_count

        if processed_count > len(processed_ids):
            elapsed_time = time.time() - monitoring_data['start_time']
            videos_processed_this_session = processed_count - len(processed_ids)
            avg_time_per_video = elapsed_time / videos_processed_this_session
            remaining_videos = len(remaining) - idx
            eta_seconds = remaining_videos * avg_time_per_video
            monitoring_data['eta_minutes'] = eta_seconds / 60
            logger.info(f"⏱️ ETA: {monitoring_data['eta_minutes']:.1f} minutes ({avg_time_per_video:.1f}s/video)")

        delay = random.uniform(MIN_DELAY, MAX_DELAY)

        if (processed_count - len(processed_ids)) % 10 == 0:
            extra_delay = random.uniform(180, 300) 
            monitoring_data['current_status'] = f"🎯 Milestone break: {extra_delay:.1f}s"
            update_progress_display()
            save_progress_report() 
            logger.info(f"🎯 Milestone break: {extra_delay:.1f}s")
            time.sleep(extra_delay)

        monitoring_data['current_status'] = f"😴 Delay: {delay:.1f}s"
        update_progress_display()
        logger.info(f"😴 Standard delay: {delay:.1f}s")
        time.sleep(delay)

except KeyboardInterrupt:
    logger.info("\n⏹️ Interrupted by user")
    monitoring_data['current_status'] = "⏹️ Interrupted by user"
    update_progress_display()

# --- Final Merge and Cleanup (Ensures ALL original columns are kept) ---

final_df_scraped = pd.DataFrame(results)

# Remove any columns from the scraped data that conflict with the original structure columns before merge
scraped_cols_to_keep = ['video_id', 'title', 'channel_name', 'view_count', 'publish_date', 'video_length', 'transcript', 'transcript_type', 'is_transcript_available']
final_df_scraped = final_df_scraped[[col for col in scraped_cols_to_keep if col in final_df_scraped.columns]]

# Perform LEFT MERGE: Keeps ALL columns from df_original.
# Use distinct suffixes to ensure new columns are added.
df_final = pd.merge(
    df_original, 
    final_df_scraped, 
    on='video_id', 
    how='left',
    suffixes=('_original', '_scraped')
)

# Rename the scraped columns to the final desired output names.
df_final.rename(columns={
    'title_scraped': 'video_title',
    'channel_name_scraped': 'video_channel_name',
    'view_count_scraped': 'video_view_count',
    'publish_date_scraped': 'video_publish_date',
    'video_length_scraped': 'video_length_seconds',
    'transcript': 'video_transcript', 
    'transcript_type': 'transcript_source', 
    'is_transcript_available': 'is_transcript_available',
}, inplace=True)

# Clean up duplicate columns resulting from the merge process
cols_to_drop = [col for col in df_final.columns if col.endswith('_original') or col in ['title', 'channel_name', 'view_count', 'publish_date', 'video_length']]
df_final.drop(columns=cols_to_drop, inplace=True, errors='ignore')

# Restore original video ID column name
if original_id_column_name != 'video_id':
     df_final.rename(columns={'video_id': original_id_column_name}, inplace=True) 

# Final save
df_final.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

# Final statistics with monitoring update
end_time = time.time()
total_time = end_time - monitoring_data['start_time']

monitoring_data['current_status'] = "🎉 Completed!"
update_progress_display()

logger.info(f"\n🎉 Process completed!")
logger.info(f"⏱️ Total runtime: {total_time/60:.1f} minutes")
logger.info(f"📊 Videos processed: {processed_count}")
logger.info(f"✅ Successful transcripts: {monitoring_data['successful_transcripts']}")
logger.info(f"❌ Videos with no transcripts: {monitoring_data['failed_videos']}")
logger.info(f"📁 Results saved to: {OUTPUT_CSV}")
if processed_count > 0:
    logger.info(f"📈 Success rate: {monitoring_data['successful_transcripts']/processed_count*100:.1f}%")

# Save final report
save_progress_report()

# Display sample results
if os.path.exists(OUTPUT_CSV):
    final_df = pd.read_csv(OUTPUT_CSV)
    logger.info(f"📊 Final CSV contains {len(final_df)} total videos")
    logger.info(f"📋 Final CSV columns: {list(final_df.columns)}") 

    if 'transcript_source' in final_df.columns:
        type_counts = final_df['transcript_source'].value_counts()
        logger.info(f"📋 Transcript types: {dict(type_counts)}")

    logger.info(f"📝 Sample results:")
    id_col_ref = original_id_column_name if original_id_column_name in final_df.columns else 'video_id'
    for idx, row in final_df.head(3).iterrows():
        title_preview = row.get('video_title', 'N/A')
        title_preview = title_preview[:40] + "..." if isinstance(title_preview, str) and len(title_preview) > 40 else title_preview
        transcript_preview = row.get('video_transcript', 'No Transcript')
        preview = transcript_preview[:100] + "..." if isinstance(transcript_preview, str) and len(transcript_preview) > 100 else transcript_preview
        transcript_available = row.get('is_transcript_available', False)
        logger.info(f"  Video ID: {row[id_col_ref]} | Title: {title_preview} | T-Available: {transcript_available} | Transcript Preview: {preview}")

print(f"\n🎯 Script completed successfully!")
print(f"📊 Final Report saved to: {os.path.join(OUTPUT_FOLDER, 'progress_report.txt')}") 
print(f"📋 Progress Log saved to: {PROGRESS_LOG}")

# Create final summary visualization (optional)
try:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    labels = ['Successful Transcript', 'No Transcript']
    sizes = [monitoring_data['successful_transcripts'], monitoring_data['failed_videos']]
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax1.set_title('Transcript Extraction Results')

    if monitoring_data['transcript_types']:
        types = monitoring_data['transcript_types']
        labels2 = []
        sizes2 = []
        for t_type, count in types.items():
            if count > 0:
                labels2.append(t_type.title())
                sizes2.append(count)

        if sizes2:
            ax2.pie(sizes2, labels=labels2, autopct='%1.1f%%', startangle=90)
            ax2.set_title('Transcript Types')

    plt.tight_layout()
    visualization_path = os.path.join(OUTPUT_FOLDER, 'results_summary.png')
    plt.savefig(visualization_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"📊 Results visualization saved to: {visualization_path}")

except Exception as e:
    print(f"⚠️ Could not create visualization: {e}")

print("\n" + "="*60)
print("🎉 EXTRACTION COMPLETE - MONITORING DATA SAVED")
print("="*60)